## Functions

In [163]:
import LyndonWords
import importlib
import AnalysisFuncs
importlib.reload(LyndonWords)
importlib.reload(AnalysisFuncs)
from AnalysisFuncs import *
from LyndonWords import *
import itertools
from functools import partial
import multiprocessing as mp
from tqdm.notebook import tqdm
import math
import numpy as np
def gen_permutations(type,n,start=[],end=[]):
    potential = list(range(n+1))
    for i in start:
        potential.remove(i)
    for i in end:
        potential.remove(i)
    for perm in itertools.permutations(np.array(potential)):
        yield rootSystem(start + list(perm) + end,type)

### Printing and Exporting

In [164]:
def print_by_length(rootSystem:rootSystem):
    len = 0
    for k in range(rootSystem.k + 1):
        for i in rootSystem.baseRoots:
            comb = i + rootSystem.delta * k
            if(sum(comb) != len):
                len = sum(comb)
                print(len)
            for i in rootSystem.SL(comb):
                print(" "+str(i))

In [165]:
def print_by_base_word(rootSystem:rootSystem,comb):
    k=0
    is_imaginary = rootSystem.is_imaginary(comb)
    for i in rootSystem.get_chain(comb):
        if(is_imaginary and k % rootSystem.n == 0):
            print(k //rootSystem.n)
        if(not is_imaginary):
            print(k)
        print(" " + str(i))
        k+=1

In [166]:
def export_by_length(rootsys:rootSystem,quoteMarks=False):
    file = open("out.txt",'w')
    file.write(str(rootsys.ordering)+"\n")
    for k in range(rootsys.k + 1):
        for i in rootsys.baseRoots:
            comb = i + rootsys.delta * k
            for i in rootsys.SL(comb):
                if(quoteMarks):
                    file.write(f"'{str(i)}',")
                else:
                    file.write(str(i) + '\n')
    file.close()

### Delta Pattern Detection

In [167]:
def write_im_words_to_file(rootSystems,file="out.txt",total=0,notOnlyAllStandard=True,k=2):
    fileobj = open(file,'w')
    fileobj.write("imaginary index, splitting type, inserted imaginary index, left standard, right standard, rotation index,bracketing\n")
    deltaTypesCol:deltaTypesCollection
    first:rootSystem
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        deltaTypesCol = generate_delta_types(first)
        if(notOnlyAllStandard):
            if(deltaTypesCol.not_all_standard()):
                fileobj.write(deltaTypesCol.to_csv())
    with mp.Pool() as p:
        for deltaTypesCol in tqdm(p.imap_unordered(generate_delta_types,rootSystems),total=total):
            if(notOnlyAllStandard):
                if(not deltaTypesCol.not_all_standard()):
                    continue
            fileobj.write(deltaTypesCol.to_csv())
    fileobj.close()

In [168]:
def check_standard_fac_same_parrallel(rootSystems,total=0,k=2):
    results = []
    if(total == 0):
        first = next(rootSystems)
        total = math.factorial(first.n+1)-1
        result = check_standard_fac_same(first)
        if(result != True):
            yield result
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_standard_fac_same,k=k),rootSystems,chunksize=5),total=total):
            if(result != True):
                yield result
    return results

### Imaginary Words Stuff

In [169]:
def check_h_deltas_repeat(perms):
    excepts = []
    i:rootSystem
    for i in perms:
        hs = [j.hs for j in i.get_chain(i.delta)]
        bases = hs[i.n:2*i.n]
        for offset in range(0,i.n):
            for k in range(2,len(hs)//i.n):
                if(np.any(bases[offset] != hs[offset + k*i.n])):
                    excepts.append((str(i.ordering),f"SL_{offset+1}({k}\delta)"))
    return excepts

In [170]:
def check_one_smallest_last(perms):
    rootsys:rootSystem
    for rootsys in perms:
        if(rootsys.delta[rootsys.ordering[0].rootIndex] == 1):
            if(rootsys.SL(rootsys.delta)[0][-1].rootIndex != rootsys.ordering[1].rootIndex):
                yield (str(rootsys.ordering),rootsys.SL(rootsys.delta)[0].noCommas(),rootsys.ordering[1].rootIndex)

### Rootsystem equivalences

In [171]:
def compare_rootsystems(r1:rootSystem,r2:rootSystem):
    words = list(r1.rootToWordDictionary.values())
    for i in words:
        for j in range(len(i)):
            if(str(i[j]) != str(r2.SL(i[j].degree)[j])):
                yield (i[j].degree,str(i[j]),str(r2.SL(i[j].degree)[j]))

In [172]:
def generate_equivalences(permutations):
    equivalences = np.arange(len(permutations))
    for i in range(len(permutations)):
        for j in range(0,i):
            if(equivalences[j] != j):
                continue
            flag = False
            for k in compare_rootsystems(permutations[i],permutations[j]):
                flag =True
                break
            if(not flag):
                yield (permutations[i].ordering,permutations[j].ordering)

### Periodicity

In [173]:
def get_max_periodicity(rootSystems,total=0):
    max = 0
    maxOrdering = ""
    maxRoot = []
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(max_periodicity_rootSystem,rootSystems),total=total):
            if(result.periodicity > max):
                max = result.periodicity
                maxOrdering = result.ordering
                maxRoot= result.maxRoot
    return (max,maxOrdering,maxRoot)

In [174]:
def verify_periodicity_perms(perms,total=1):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(verify_periodicity,perms),total=total):
            if not result:
                print("Exception")

In [175]:
def check_basic_periodicity_perms(perms,total,k=1):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_basic_periodicity,k=k),perms),total=total):
            if(result is not None):
                yield result

In [176]:
def periodicity_ell_1_perms(perms,total=1):
    with mp.Pool() as p:
        for i in tqdm(p.imap_unordered(periodicity_ell_1,perms),total=total):
            if i is None:
                continue
            print(i)

### Convexity

In [177]:
def check_convexity_perms(perms,total=0,k=5,word_convexity=False):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(convexity_from_perm,k=k,word_convexity=word_convexity),perms),total=total):
            if(result is not None):
                yield result

In [178]:
def check_min_right_factor_gamma_im(rootsys:rootSystem,n=5):
    for delta_val in range(n):
        for base_root in rootsys.baseRoots[:-1]:
            if(delta_val == 0 and sum(base_root) == 1):
                continue
            w:word = rootsys.SL(base_root + rootsys.delta * delta_val)[0]
            rightstand:word = rootsys.standfac(w)[1]
            min_right = rootsys.get_min_right_factor(w.degree)
            if sum(rightstand.degree - min_right.degree) == 0:
                continue
            if(rootsys.is_imaginary_height(sum(rightstand.degree - min_right.degree))):
                print(f"{str(rootsys.ordering)}: {base_root}: {delta_val}: {rightstand.degree}: {min_right.degree}")

In [179]:
def check_equiv_of_ell_on_standard_fac(rootsys:rootSystem):
    rootsys.generate_up_to_delta(1)
    for w in rootsys.baseRoots[:-1]:
        ell = rootsys.max_im_word(w)
        standw = rootsys.standfac(ell)[0].degree        
        standhs = (standw - (rootsys.delta *standw[0]))
        standhs = -standhs[1:]
        if(not rootsys.split_e_bracket(standhs,rootsys.SL(w)[0])):
            print(f"Exception: {rootsys.ordering}, {ell.no_commas()}")

In [180]:
def check_intersection_im(rootsys:rootSystem,alpha):
    for d in rootsys.get_decompositions(alpha):
        if(rootsys.is_imaginary_height(sum(d[0]))
           or rootsys.is_imaginary_height(sum(d[1]))):
            continue
        s1 = rootsys.get_imaginary_convex_set(d[0])
        s2 = rootsys.get_imaginary_convex_set(d[1])
        flag = False
        for s in s1:
            for p in s2:
                if s ==p:
                    flag = True
                    break
            if(flag):
                break
        if(not flag):
            print(d[0])
            print(d[1])
            print(*[i.no_commas() for i in s1])
            print(*[i.no_commas() for i in s2])

### Chains

In [181]:
def check_chain_bound_by_m(rootsys:rootSystem,root,k=1):
    rootsys.generate_up_to_delta(k)
    chain = rootsys.get_chain(root)
    convexset  = rootsys.get_imaginary_convex_set(root)
    minconvex = convexset[-1]
    maxconvex = convexset[0]
    for i in chain:
        if(maxconvex < i):
            return True
        if minconvex < i:
            return False
    return True
    

In [182]:
def check_chain_bound_rootsys(rootsys:rootSystem,genheight=3,k=1):
    rootsys.generate_up_to_delta(genheight)
    for i in rootsys.baseRoots[:-1]:
        if(not check_chain_bound_by_m(rootsys,i,k)):
            yield i

In [183]:
def check_irr_chain_conj_perms(perms,total=0,k=3):
    with mp.Pool() as p:
        for result in tqdm(p.imap_unordered(partial(check_irr_chains_conj,k=k),perms),total=total):
            if result is not None:
                print(result)

## Type A

In [114]:
n=6
Aperms = gen_permutations('A',n)

In [65]:
verify_periodicity_perms(Aperms,math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
write_im_words_to_file(Aperms,"ADeltaTypes.csv",math.factorial(n+1))

  0%|          | 0/720 [00:00<?, ?it/s]

In [57]:
get_max_periodicity(Aperms)

0it [00:00, ?it/s]

IndexError: only integers, slices (`:`), ellipsis (`...`), numpy.newaxis (`None`) and integer or boolean arrays are valid indices

In [113]:
check_irr_chains_conj(rootSystem([0,1,2,3],'A'))

In [115]:
check_irr_chain_conj_perms(Aperms,math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

## Type C

In [21]:
n=4
Cperms = gen_permutations('C',n)

In [22]:
verify_periodicity_perms(Cperms,math.factorial(n+1))

  0%|          | 0/120 [00:00<?, ?it/s]

KeyError: 2

In [ ]:
for i in check_delta_type_prediction_perms(Cperms):
    print(i)

In [281]:
generate_delta_types(rootSystem([0,1,2,3,4,5,6],'C'))

[1,0,2] 1


In [ ]:
for n in range(2,6):
    Cperms = gen_permutations('C',n)
    write_im_words_to_file(Cperms,f"C{n}ImWords.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/719 [00:00<?, ?it/s]

  0%|          | 0/5039 [00:00<?, ?it/s]

  0%|          | 0/40319 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
for i in generate_equivalences(Cperms):
    print(f"{i[0]} = {i[1]}")

In [ ]:
get_max_periodicity(Cperms)

(5, '0<3<1<4<2', array([0, 0, 2, 2, 1]))

## Type G

In [185]:
n=2
G2perms = gen_permutations('G',n)

In [416]:
for G2 in G2perms:
    G2.verify_periodicity()

In [ ]:
get_max_periodicity(G2perms)

(5, '1<0<2', array([1, 1, 3]))

In [ ]:
for i in check_delta_type_prediction_perms(G2perms):
    print(i)

In [ ]:
write_im_words_to_file(G2perms,"GDeltaTypes.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

In [226]:
for i in check_convexity_perms(G2perms,math.factorial(n+1),k=10,word_convexity=True):
    print(i)

  0%|          | 0/6 [00:00<?, ?it/s]

## Type B

In [57]:
n=6
Bperms = gen_permutations('B',n)

In [58]:
verify_periodicity_perms(Bperms,math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
for i in check_delta_type_prediction_perms(Bperms):
    print(i)

In [ ]:
for n in range(2,7):
    Bperms = gen_permutations('B',n)
    write_im_words_to_file(Bperms,f"B{n}ImWords.csv")

  0%|          | 0/5 [00:00<?, ?it/s]

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/719 [00:00<?, ?it/s]

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
write_im_words_to_file(Bperms,"BDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
for i in generate_equivalences(Bperms):
    print(f"{i[0]} = {i[1]}")

## Type D

In [61]:
n=6
Dperms = gen_permutations('D',n)

In [62]:
verify_periodicity_perms(Dperms,math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

In [60]:
for D in tqdm(Dperms,total=math.factorial(n+1)):
    D.verify_periodicity()

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
for i in check_delta_type_prediction_perms(Dperms):
    print(i)

In [ ]:
for n in range(3,7):
    Dperms = gen_permutations('D',n)
    write_im_words_to_file(Dperms,f"D{n}ImWords.csv")

  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/119 [00:00<?, ?it/s]

  0%|          | 0/719 [00:00<?, ?it/s]

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
write_im_words_to_file(Dperms,"DDeltaTypes.csv")

  0%|          | 0/5039 [00:00<?, ?it/s]

In [ ]:
for i in generate_equivalences(Dperms):
    print(f"{i[0]} = {i[1]}")

## Type E

In [47]:
n=6
Eperms = gen_permutations('E',n)

In [44]:
for i in range(9):
    E8perms = gen_permutations('E',8,[i])
    write_im_words_to_file(E8perms,f'E8ImWords{i}start.csv')

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

  0%|          | 0/362879 [00:00<?, ?it/s]

In [208]:
periodicity_ell_1_perms(Eperms,math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

In [29]:
verify_periodicity_perms(Eperms,total=math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
write_im_words_to_file(Eperms,"E7ImWords.csv")

  0%|          | 0/40319 [00:00<?, ?it/s]

In [48]:
check_irr_chain_conj_perms(Eperms,total=math.factorial(n+1))

  0%|          | 0/5040 [00:00<?, ?it/s]

In [ ]:
get_max_periodicity(Eperms)

  0%|          | 0/5040 [00:00<?, ?it/s]

(10, '2<0<1<4<6<5<3', array([1, 0, 1, 1, 1, 1, 0]))

## Type F

In [138]:
n=4
Fperms = gen_permutations('F',4)

In [183]:
for F4 in tqdm(Fperms,total=math.factorial(n+1)):
    periodicity_ell_1(F4)

  0%|          | 0/120 [00:00<?, ?it/s]

In [27]:
for F4 in tqdm(Fperms,total=math.factorial(n+1)):
    F4.verify_periodicity()

  0%|          | 0/120 [00:00<?, ?it/s]

In [175]:
check_standard_fac_same_parrallel(Fperms,k=3)

<generator object check_standard_fac_same_parrallel at 0x00000152BBF87CD0>

In [176]:
for i in check_delta_type_prediction_perms(Fperms):
    print(i)

In [ ]:
write_im_words_to_file(Fperms,"FDeltaTypes.csv",k=2,total=math.factorial(n+1))

  0%|          | 0/120 [00:00<?, ?it/s]

In [178]:
get_max_periodicity(Fperms,math.factorial(5))

  0%|          | 0/120 [00:00<?, ?it/s]

(0, '', [])

In [ ]:
check_irr_chain_conj_perms(Fperms,math.factorial(n+1))

  0%|          | 0/120 [00:00<?, ?it/s]